Only for the optimization problem about $k$

In [24]:
import numpy as np
from scipy.optimize import Bounds, minimize
import torch
from pickle import dump

x = torch.randn(1,3,224,224)
B, C, H, W = x.shape
print(B, C, H, W)

def save_object(obj, path):
    with open(path, 'wb') as file:
        dump(obj, file)


1 3 224 224


In [25]:
def objective_function(k, *args):
    # k = x
    n, h1, h2, h3, h4 = args
    return h1 * k + h2 / k + h3 / k * np.log(n / (n - k)) + h4*n * np.log(n / (n - k))

def solve_k_scipy(n, system_params, conv_params):  # 目前是nk
    B, C_i, H_i, W_i, C_o, H_o, W_o, kernel_size, stride, padding = conv_params
    mu_m, theta_m, mu_rec, theta_rec, mu_cmp, theta_cmp, mu_sen, theta_sen = system_params

    Iov = np.prod([C_i, H_i, kernel_size - stride])
    IW = np.prod([C_i, H_i, W_o, stride])
    O = np.prod([C_o, H_o, W_o])
    Nc = np.prod([2.0, C_o, H_o, C_i, kernel_size, kernel_size, W_o]) # 这个用int32表示会溢出了

    # h_1 = np.prod(2*(1/mu_m+theta_m)*(n*Iov+O))
    # h_2 = np.prod(4.0, IW, theta_rec) + np.prod(4.0, O, theta_sen) + np.prod(Nc, theta_cmp)
    # h_3 = np.prod(4.0, IW, 1/mu_rec) + np.prod(4.0, O, 1/mu_sen) + Nc / mu_cmp
    # h_4 = np.prod(4.0, Iov, mu_rec)
    h_1 = 2.0*(1/mu_m+theta_m)*(n*Iov+O)
    h_2 = 4.0*IW*theta_rec + 4.0*O*theta_sen + Nc*theta_cmp
    h_3 = 4.0*IW/mu_rec + 4.0*O/mu_sen + Nc / mu_cmp
    h_4 = 4.0*Iov/mu_rec

    print(h_1, h_2, h_3, h_4)

    # constraints = ({'type': 'ineq', 'fun': lambda x: x[0]}) # n >= k
    bounds = Bounds([2], [n-0.5])

    initial_guess = [n/2]
    result = minimize(objective_function, initial_guess, args=(n, h_1, h_2, h_3, h_4), constraints=[], bounds=bounds)
    print(f'k*={result.x}   T*={result.fun}')

    if not result.success:
        print(result.message)
    return (result.fun, *result.x)

In [26]:
a = np.asarray([1,2,3,4,5,6])
np.searchsorted(a, 4.5)

4

In [27]:
def get_idxes(src_array: list, selected_array):
    return [src_array.index(i) for i in selected_array]

vgg16_conv_idxes = [0, 1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
resnet_conv_idxes = [0, 3, 6, 10, 13, 17, 20, 22, 26, 29, 33, 36, 38, 42, 45, 49, 52, 54, 58, 61]

### Sending testing

In [28]:
N_tr = 4 * np.prod((1,10,224,224))
# lambda=0
# drop_ratio=0.0,mu_theta=[1.49225417 0.93664279]
# drop_ratio=0.6,mu_theta=[10.49168505  1.48482727]
# drop_ratio=0.7,mu_theta=[9.98609349 1.47616386]
# drop_ratio=0.8,mu_theta=[16.47243177  1.56259472]
# drop_ratio=0.9,mu_theta=[62.69922425  1.68099049]√
# Mean=1.4425667207241057, Var=0.08658871419649657

# lambda=2
# drop_ratio=0.0,mu_theta=[0.97334611 0.58018312]
# drop_ratio=0.6,mu_theta=[6.36061955 1.43551055]
# drop_ratio=0.8,mu_theta=[17.11664517  1.62201568]√
# Mean=1.3571005320549012, Var=0.16216833022643398

# lambda=4
# drop_ratio=0.0,mu_theta = [0.95102672, 0.46027601]
# Mean=1.325169739484787, Var=0.3967943469793664
# drop_ratio=0.6,mu_theta = [2.31140114, 1.09813618]√
# Mean=1.3717069041728973, Var=0.4536102989240614

# lambda=6
# drop_ratio=0.0,mu_theta = [0.8022083, 0.44171801]
# drop_ratio=0.0,mu_theta=[0.81696626 0.43643751]
# drop_ratio=0.6,mu_theta=[1.19833621 0.73893584]√
# Mean=1.539291063785553, Var=0.9629890057676237
# Mean=1.5666165959835052, Var=1.0540162402143467

# drop_ratio=0.0,mu_theta=[0.8060863  0.43355355]
# drop_ratio=0.6,mu_theta=[1.16349416 0.71954717]
# drop_ratio=0.7,mu_theta=[1.10985499 0.64714996]
# drop_ratio=0.8,mu_theta=[1.0534411  0.54543782]
# drop_ratio=0.9,mu_theta=[1.15052917 0.7453204 ]
# Mean=1.5348685916264853, Var=0.9016922974699393

# lambda=8
# drop_ratio=0.0,mu_theta=[0.66294479 0.43224486]
# drop_ratio=0.6,mu_theta=[0.6873872 0.34175332]
# Mean=1.8839759855270386, Var=1.9697738570009067

# drop_ratio=0.0,mu_theta=[0.67035402 0.41059232]
# drop_ratio=0.6,mu_theta=[0.72084543 0.43649281]
# drop_ratio=0.7,mu_theta=[0.69784125 0.35931745]
# drop_ratio=0.8,mu_theta=[0.70131561 0.37896212]√
# drop_ratio=0.9,mu_theta=[ 0.62072898 -0.1186138 ]
# Mean=1.860271238565445, Var=2.1201451106325817

# lambda=10
# drop_ratio=0.0,mu_theta=[0.56154803 0.40367499]
# drop_ratio=0.6,mu_theta=[0.59548332 0.38495174]
# Mean=2.1276258549690246, Var=2.9237235769880963

# drop_ratio=0.0,mu_theta=[0.62175678 0.43279159]
# drop_ratio=0.6,mu_theta=[0.61142539 0.27758693]√
# drop_ratio=0.7,mu_theta=[ 0.54826549 -0.04835925]
# drop_ratio=0.8,mu_theta=[ 0.53528021 -0.13041395]
# drop_ratio=0.9,mu_theta=[ 0.54078816 -0.07153885]
# Mean=2.0390875606536865, Var=2.7433485655585437

mu_sen = 16.47243177 * N_tr
theta_sen = 1.56259472 / N_tr

# len=6: lambda={0, 2, 4, 6, 8, 10}
mu_theta_sen_pairs = [(62.69922425, 1.68099049), (17.11664517, 1.62201568), (2.31140114, 1.09813618), (1.19833621, 0.73893584), (0.70131561, 0.37896212), (0.61142539, 0.27758693)]


### Receiving test

In [29]:
# drop_ratio=0.0,mu_theta=[2.02444181 1.19248489]
# drop_ratio=0.6,mu_theta=[21.52334744  1.60510338]
# drop_ratio=0.7,mu_theta=[28.78313455  1.62612959]
# drop_ratio=0.8,mu_theta=[46.34628024  1.65427234]
# drop_ratio=0.9 mu_theta=[133.28784691, 1.68944582]√
# Mean=1.5534600559473037, Var=0.025916803601791667
# Means=[1.58102112 1.5114809  1.58212576 1.5413563  1.50508581 1.52142071
#  1.59866629 1.52320613 1.56526118 1.60497636]
# Vars=[0.021124   0.02540957 0.02421154 0.02407216 0.033022   0.03280923
#  0.01684095 0.0372001  0.01716482 0.0147049 ]

# drop_ratio=0.0,mu_theta=[2.13327949 1.20315024]
# drop_ratio=0.6,mu_theta=[17.26289812  1.58102757]
# drop_ratio=0.7,mu_theta=[23.27839373  1.6078966 ]
# drop_ratio=0.8,mu_theta=[42.39740938  1.64872724]
# drop_ratio=0.9,mu_theta=[102.67279512   1.68352874]
# Mean=1.5472359682321548, Var=0.025083978038832604
# Means=[1.54813835 1.51226388 1.50610318 1.59812899 1.55416482 1.53098486
#  1.54865413 1.55407425 1.56389815 1.55594908]
# Vars=[0.03279974 0.02672232 0.03649977 0.0244112  0.02143169 0.02651675
#  0.01923343 0.01682472 0.01544599 0.02473387]

mu_theta_rec_pair = (133.28784691, 1.68944582)

In [30]:
B = 1
# input shape
C_i, H_i, W_i = input_shape = 64, 224, 224
# output shape
C_o, H_o, W_o = output_shape = 64, 224, 224

print(np.prod(input_shape))

kernel_size, stride, padding = 3, 1, 1

conv_params = (B, C_i, H_i, W_i, C_o, H_o, W_o, kernel_size, stride, padding)

# scale = 100000  # shape (scale, n) for generating random variables, n values per group

# mu_m, theta_m = 1375069010.349571, 1.1414616840348712e-09
mu_m, theta_m = 16904242506.376558, 1.5748831477819342e-09

# mu_sen, theta_sen = (664446.2862308504, 7.829987338890e-07)
mu_sen, theta_sen = (16.47243177 * N_tr, 1.56259472 / N_tr)
# mu_rec, theta_rec = (1196567.3770074411, 3.3175139905527e-07)
mu_rec, theta_rec = (133.28784691 * N_tr, 1.68944582 / N_tr)
mu_cmp, theta_cmp = (16904242506.376558, 1.5748831477819342e-09)  

system_params = mu_m, theta_m, mu_rec, theta_rec, mu_cmp, theta_cmp, mu_sen, theta_sen

objective, k = solve_k_scipy(10, system_params, conv_params)
print(objective, k)

3211264
0.011431690815786365 26.639144577293983 0.6553873800496556 0.00042871768482719756
k*=[9.5]   T*=3.1322347918903226
3.1322347918903226 9.5


In [31]:
# determine k* by layer according to given system parameters and conv parameters

VGG16

In [47]:
# lambda=0
# Results of single method:
# 'coded-3': {'min':12.907898902893066, 'mean':13.579340362548828, 'max':14.862844705581665}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-2': {'min':11.46385669708252, 'mean':12.034688401222226, 'max':12.568840980529785}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-1': {'min':10.626862049102783, 'mean':11.173993277549743, 'max':11.807914972305298}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# Results of mixed method:
# Layer 0: [1.59021623 1.39210598 1.21150393]
# Layer 1: [3.14448869 2.82892601 2.50694578]
# Layer 3: [1.11541286 1.02091546 0.87412131]
# Layer 4: [1.69060264 1.47511744 1.2429251 ]
# Layer 6: [0.60760131 0.57369993 0.50419986]
# Layer 7: [1.04891253 0.98820601 0.89389977]
# Layer 8: [1.06890693 0.97000344 0.87080948]
# Layer 10: [0.48210137 0.39249954 0.39580123]
# Layer 11: [1.16310189 0.97800417 1.02981758]
# Layer 12: [1.18891973 0.96371646 0.98840714]
# Layer 14: [0.57450206 0.50569963 0.51200392]
# Layer 15: [0.57960038 0.49720411 0.50335796]
# Layer 16: [0.57039928 0.49590549 0.50691345]
# Conv latency: [0.319610595703125, 2.506945776939392, 0.8741213083267212, 1.2429250955581665, 0.5041998624801636, 0.8938997745513916, 0.8708094835281373, 0.3924995422363281, 0.9780041694641113, 0.963716459274292, 0.505699634552002, 0.4972041130065918, 0.4959054946899414]
# Hybrid minimum of mean layer latency: 11.070721411705016 ???

# Results of single method:
# 'coded-2': {'min':11.855223178863525, 'mean':12.29489505290985, 'max':13.64164924621582}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-1': {'min':11.174071311950684, 'mean':11.507628520329792, 'max':12.14154601097107}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# Results of mixed method:
# Layer 0: [1.41989567 1.21504591]
# Layer 1: [2.98283329 2.81179182]
# Layer 3: [1.02298168 0.8896426 ]
# Layer 4: [1.51116377 1.27667691]
# Layer 6: [0.5634801  0.50122903]
# Layer 7: [0.99466004 0.84646664]
# Layer 8: [0.99736706 0.87846311]
# Layer 10: [0.39023447 0.39833524]
# Layer 11: [0.99709862 1.03523497]
# Layer 12: [0.99253635 0.99780852]
# Layer 14: [0.50011637 0.51409642]
# Layer 15: [0.49944381 0.50553183]
# Layer 16: [0.49818879 0.50756075]
# Conv latency: [0.319610595703125, 2.8117918173472085, 0.889642596244812, 1.2766769131024678, 0.5012290279070536, 0.8464666406313578, 0.8784631093343099, 0.39023447036743164, 0.9970986247062683, 0.9925363461176554, 0.5001163681348165, 0.4994438091913859, 0.4981887936592102]
# Hybrid minimum of mean layer latency: 11.426679213841757

vgg16_latency_lambda0 = np.asarray(
    [[1.59021623, 1.39210598, 1.21150393],
    [3.14448869, 2.82892601, 2.50694578],
    [1.11541286, 1.02091546, 0.87412131],
    [1.69060264, 1.47511744, 1.2429251 ],
    [0.60760131, 0.5634801,  0.50122903],
    [1.04891253, 0.99466004, 0.84646664],
    [1.06890693, 0.97000344, 0.87080948],
    [0.48210137, 0.39249954, 0.39580123],
    [1.16310189, 0.97800417, 1.02981758],
    [1.18891973, 0.96371646, 0.98840714],
    [0.57450206, 0.50569963, 0.51200392],
    [0.57960038, 0.49720411, 0.50335796],
    [0.57039928, 0.49590549, 0.50691345]])

distributed_conv_layers = [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
distributed_conv_idxes = get_idxes(vgg16_conv_idxes, distributed_conv_layers)
records = vgg16_latency_lambda0[distributed_conv_idxes]
ks = records.argmin(axis=-1) + 7
print(list(ks))


[9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8]


In [48]:
# lambda=2
# Results of single method:
# 'coded-3': {'min':12.739831924438477, 'mean':13.861861089865366, 'max':17.106017112731934}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-2': {'min':11.44785737991333, 'mean':12.096827387809753, 'max':13.622917890548706}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-1': {'min':10.747905015945435, 'mean':11.436567823092142, 'max':12.412842988967896}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# Results of mixed method:
# Layer 0: [1.5933339  1.40059133 1.30701192]
# Layer 1: [3.28159624 2.78945937 2.50709546]
# Layer 3: [1.13984036 1.00958524 0.89508855]
# Layer 4: [1.62527329 1.46526136 1.29167249]
# Layer 6: [0.64475896 0.57526124 0.51433615]
# Layer 7: [1.02508646 0.96950843 0.91059472]
# Layer 8: [1.0553508  0.98116785 0.86592277]
# Layer 10: [0.49992839 0.40116276 0.43000066]
# Layer 11: [1.21117723 1.0043336  1.04360304]
# Layer 12: [1.25125962 1.01269788 1.05617032]
# Layer 14: [0.59429264 0.51058632 0.52610149]
# Layer 15: [0.59142049 0.51450404 0.52816935]
# Layer 16: [0.59708589 0.51850861 0.52302212]
# Conv latency: [0.319610595703125, 2.507095456123352, 0.8950885534286499, 1.2916724880536397, 0.5143361488978068, 0.9105947216351827, 0.865922768910726, 0.4011627634366353, 1.0043335954348247, 1.0126978754997253, 0.5105863213539124, 0.5145040353139242, 0.5185086131095886]
# Hybrid minimum of mean layer latency: 11.291294038295746

# Results of single method:
# 'coded-3': {'min':12.650034666061401, 'mean':13.467434247334797, 'max':14.424841403961182}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-2': {'min':11.396790027618408, 'mean':12.066825032234194, 'max':12.716983318328857}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-1': {'min':10.677037954330444, 'mean':11.282983322938282, 'max':11.912921667098999}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# Results of mixed method:
# Layer 0: [1.58150373 1.39067374 1.21000135]
# Layer 1: [3.1212947  2.88042881 2.56911151]
# Layer 3: [1.13633434 1.01675848 0.90701662]
# Layer 4: [1.58511849 1.43550942 1.34141791]
# Layer 6: [0.63766756 0.58084057 0.50000044]
# Layer 7: [1.02626451 0.98962426 0.84608678]
# Layer 8: [1.06308371 0.96401457 0.8545176 ]
# Layer 10: [0.48750035 0.38916655 0.40250415]
# Layer 11: [1.17010488 0.97942485 1.00527094]
# Layer 12: [1.18076388 0.98867071 0.98884362]
# Layer 14: [0.57084354 0.4979163  0.50475723]
# Layer 15: [0.57783441 0.50208793 0.51150177]
# Layer 16: [0.56583319 0.49759187 0.50716408]
# Conv latency: [0.319610595703125, 2.5691115061442056, 0.9070166150728861, 1.341417908668518, 0.5000004371007284, 0.8460867802302042, 0.8545175989468893, 0.3891665538152059, 0.9794248541196188, 0.9886707067489624, 0.497916301091512, 0.502087930838267, 0.4975918730099996]
# Hybrid minimum of mean layer latency: 11.217799762884777

# Results of single method:
# 'coded-2': {'min':11.49965214729309, 'mean':12.012837251027426, 'max':12.597570657730103}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-1': {'min':10.711998462677002, 'mean':11.313208599885304, 'max':11.773922204971313}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# Results of mixed method:
# Layer 0: [1.40605396 1.22279098]
# Layer 1: [2.83448756 2.65937016]
# Layer 3: [1.01115723 0.87511635]
# Layer 4: [1.50815469 1.31303459]
# Layer 6: [0.57009592 0.50323915]
# Layer 7: [0.94120332 0.86411723]
# Layer 8: [0.96721768 0.83826588]
# Layer 10: [0.37935619 0.39095807]
# Layer 11: [0.97769994 0.99837283]
# Layer 12: [0.9802415  0.99216674]
# Layer 14: [0.49709886 0.50274599]
# Layer 15: [0.49556893 0.50552501]
# Layer 16: [0.50576474 0.5255059 ]
# Conv latency: [0.319610595703125, 2.659370164076487, 0.8751163482666016, 1.3130345940589905, 0.5032391548156738, 0.8641172250111898, 0.8382658759752909, 0.3793561855951945, 0.9776999354362488, 0.9802414973576864, 0.4970988631248474, 0.4955689311027527, 0.5057647426923116]
# Hybrid minimum of mean layer latency: 11.233664214611053

vgg16_latency_lambda2 = np.asarray(
    [[1.5933339, 1.40059133, 1.22279098],
    [3.28159624, 2.78945937, 2.50709546],
    [1.13984036, 1.00958524, 0.87511635],
    [1.62527329, 1.46526136, 1.29167249],
    [0.64475896, 0.57526124, 0.50323915],
    [1.02508646, 0.96950843, 0.86411723],
    [1.0553508,  0.98116785, 0.86592277],
    [0.49992839, 0.40116276, 0.39095807],
    [1.21117723, 1.0043336,  1.04360304],
    [1.25125962, 1.01269788, 1.05617032],
    [0.59429264, 0.51058632, 0.52610149],
    [0.59142049, 0.51450404, 0.52816935],
    [0.59708589, 0.51850861, 0.52302212]])

distributed_conv_layers = [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
distributed_conv_idxes = get_idxes(vgg16_conv_idxes, distributed_conv_layers)
records = vgg16_latency_lambda2[distributed_conv_idxes]
ks = records.argmin(axis=-1) + 7
print(list(ks))

[9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8]


In [49]:
# lambda=4
# Results of single method:
# 'coded-3': {'min':12.409835815429688, 'mean':13.495242357254027, 'max':16.14510488510132}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-2': {'min':11.14687466621399, 'mean':12.210952043533327, 'max':14.67179822921753}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-1': {'min':10.782899141311646, 'mean':12.090739786624912, 'max':14.466876983642578}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# Results of mixed method:
# Layer 0: [1.59484535 1.39434127 1.22276364]
# Layer 1: [3.0756282  2.87070831 2.97705521]
# Layer 3: [1.12726239 1.06717829 0.88808912]
# Layer 4: [1.6513501  1.47867336 1.34617388]
# Layer 6: [0.62033721 0.60900084 0.54775383]
# Layer 7: [1.08101098 0.98408502 0.95050089]
# Layer 8: [1.05384878 0.92534806 0.84300331]
# Layer 10: [0.48966595 0.42091924 0.47750602]
# Layer 11: [1.15719352 0.98880229 1.10751837]
# Layer 12: [1.10738502 0.97526427 1.01316746]
# Layer 14: [0.59912592 0.52166953 0.53992345]
# Layer 15: [0.58167201 0.50950891 0.54008414]
# Layer 16: [0.60597157 0.51500322 0.5151734 ]
# Conv latency: [0.319610595703125, 2.8707083066304526, 0.888089120388031, 1.346173882484436, 0.5477538307507833, 0.9505008856455485, 0.8430033127466837, 0.4209192395210266, 0.9888022939364115, 0.9752642711003622, 0.5216695268948873, 0.5095089077949524, 0.5150032242139181]
# Hybrid minimum of mean layer latency: 11.722187499205273

# Results of single method:
# 'coded-3': {'min':12.286977291107178, 'mean':13.13655404249827, 'max':14.858262538909912}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-2': {'min':11.170728921890259, 'mean':11.966281414031984, 'max':13.464102506637573}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-1': {'min':10.75290822982788, 'mean':11.372695863246918, 'max':12.672716856002808}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# Results of mixed method:
# Layer 0: [1.59000228 1.39991738 1.22641734]
# Layer 1: [2.9018754  2.73638107 2.54436634]
# Layer 3: [1.10350428 1.01233985 0.8974176 ]
# Layer 4: [1.53551284 1.51531386 1.33424197]
# Layer 6: [0.65760946 0.58566747 0.52350003]
# Layer 7: [1.02330027 0.93318117 0.89892481]
# Layer 8: [1.06333939 0.97169316 0.84600691]
# Layer 10: [0.48000256 0.38724991 0.41515265]
# Layer 11: [1.14429837 0.97306657 0.98991815]
# Layer 12: [1.134456   0.99085367 1.01094637]
# Layer 14: [0.59200559 0.50334106 0.52558299]
# Layer 15: [0.58426142 0.51333555 0.52300966]
# Layer 16: [0.57159775 0.49906739 0.51883769]
# Conv latency: [0.319610595703125, 2.5443663398424783, 0.8974176049232483, 1.3342419664065044, 0.5235000252723694, 0.8989248077074686, 0.8460069100062052, 0.3872499068578084, 0.9730665683746338, 0.9908536672592163, 0.5033410588900248, 0.5133355458577474, 0.49906738599141437]
# Hybrid minimum of mean layer latency: 11.256162484486898

vgg16_latency_lambda4 = np.asarray(
    [[1.59484535, 1.39434127, 1.22276364],
    [2.9018754,  2.73638107, 2.54436634],
    [1.12726239, 1.06717829, 0.88808912],
    [1.53551284, 1.51531386, 1.33424197],
    [0.65760946, 0.58566747, 0.52350003],
    [1.02330027, 0.93318117, 0.89892481],
    [1.05384878, 0.92534806, 0.84300331],
    [0.48000256, 0.38724991, 0.41515265],
    [1.14429837, 0.97306657, 0.98991815],
    [1.10738502, 0.97526427, 1.01316746],
    [0.59912592, 0.52166953, 0.53992345],
    [0.58426142, 0.51333555, 0.52300966],
    [0.60597157, 0.51500322, 0.5151734 ]])

distributed_conv_layers = [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
distributed_conv_idxes = get_idxes(vgg16_conv_idxes, distributed_conv_layers)
records = vgg16_latency_lambda4[distributed_conv_idxes]
ks = records.argmin(axis=-1) + 7
print(list(ks))

[9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8]


In [50]:
# lambda=6
# Results of single method:
# 'coded-3': {'min':12.649161577224731, 'mean':14.975890417893728, 'max':21.609859228134155}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-2': {'min':11.4899001121521, 'mean':13.305021365483599, 'max':18.008302688598633}
# Distributed conv idxes: [3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-1': {'min':10.939956188201904, 'mean':12.818941553433737, 'max':15.781945943832397}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# Results of mixed method:
# Layer 0: [1.58689574 1.40471768 1.22645575]
# Layer 1: [3.45263201 3.43763252 2.81920165]
# Layer 3: [1.18168813 0.94351407 0.90124784]
# Layer 4: [2.16534408 1.56932217 1.66796078]
# Layer 6: [0.66176488 0.60227789 0.53424623]
# Layer 7: [1.27222619 1.15116588 1.04834282]
# Layer 8: [1.16067733 1.08785913 1.11751105]
# Layer 10: [0.52391932 0.46233288 0.46783429]
# Layer 11: [1.19800518 1.0262597  1.08124987]
# Layer 12: [1.20549196 1.09192522 1.15918231]
# Layer 14: [0.59168305 0.5252646  0.55026015]
# Layer 15: [0.60741683 0.52567271 0.54116788]
# Layer 16: [0.61025075 0.53700387 0.58594598]
# Conv latency: [0.319610595703125, 2.819201648235321, 0.9012478391329447, 1.5693221688270569, 0.5342462261517843, 1.0483428239822388, 1.0878591338793437, 0.46233288447062176, 1.0262597004572551, 1.0919252236684163, 0.5252646009127299, 0.525672713915507, 0.5370038747787476]
# Hybrid minimum of mean layer latency: 12.473469535509743

# Results of single method:
# 'coded-3': {'min':12.349920749664307, 'mean':14.226960460344952, 'max':17.808924436569214}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-2': {'min':11.314836502075195, 'mean':13.127060194810234, 'max':18.068986892700195}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-1': {'min':10.814834833145142, 'mean':12.419092317422232, 'max':15.969950437545776}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# Results of mixed method:
# Layer 0: [1.58609553 1.39875104 1.22342265]
# Layer 1: [3.36551815 3.32553629 2.99519698]
# Layer 3: [1.09984837 1.0427502  0.88192586]
# Layer 4: [1.84575129 1.66275773 1.58941859]
# Layer 6: [0.64775012 0.64650337 0.55891774]
# Layer 7: [1.21818207 0.99275243 0.97692005]
# Layer 8: [1.11266881 1.14118572 0.93902868]
# Layer 10: [0.47414704 0.41324957 0.44108297]
# Layer 11: [1.19858934 1.00084231 1.05491857]
# Layer 12: [1.14734904 0.99734622 1.03525631]
# Layer 14: [0.58375706 0.51600313 0.52884213]
# Layer 15: [0.58801482 0.5172536  0.53854312]
# Layer 16: [0.60059365 0.52608891 0.53425062]
# Conv latency: [0.319610595703125, 2.995196978251139, 0.8819258610407511, 1.5894185900688171, 0.5589177409807841, 0.9769200483957926, 0.9390286803245544, 0.41324957211812335, 1.000842312971751, 0.9973462224006653, 0.5160031318664551, 0.517253597577413, 0.5260889132817587]
# Hybrid minimum of mean layer latency: 12.256982346375784

vgg16_latency_lambda6 = np.asarray(
    [[1.58689574, 1.40471768, 1.22645575],
    [3.45263201, 3.43763252, 2.81920165],
    [1.18168813, 0.94351407, 0.90124784],
    [1.84575129, 1.66275773, 1.58941859],
    [0.66176488, 0.60227789, 0.53424623],
    [1.21818207, 0.99275243, 0.97692005],
    [1.11266881, 1.14118572, 0.93902868],
    [0.47414704, 0.41324957, 0.44108297],
    [1.19858934, 1.00084231, 1.05491857],
    [1.14734904, 0.99734622, 1.03525631],
    [0.58375706, 0.51600313, 0.52884213],
    [0.58801482, 0.5172536,  0.53854312],
    [0.60059365, 0.52608891, 0.53425062]])

distributed_conv_layers = [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
distributed_conv_idxes = get_idxes(vgg16_conv_idxes, distributed_conv_layers)
records = vgg16_latency_lambda6[distributed_conv_idxes]
ks = records.argmin(axis=-1) + 7
print(list(ks))

[9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8]


In [51]:
# lambda=8
# Results of single method:
# 'coded-3': {'min':12.821915626525879, 'mean':16.660992840925854, 'max':22.111902475357056}
# Distributed conv idxes: [3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-2': {'min':11.390907764434814, 'mean':14.636705656846363, 'max':21.540908336639404}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-1': {'min':11.497905492782593, 'mean':14.538773755232496, 'max':20.188974618911743}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# Results of mixed method:
# Layer 0: [1.5944193  1.39893893 1.22967013]
# Layer 1: [4.75798122 3.44498706 3.1058683 ]
# Layer 3: [1.21101264 1.19642287 1.03876215]
# Layer 4: [2.10464825 2.23761666 1.94252646]
# Layer 6: [0.80565492 0.6529246  0.72408827]
# Layer 7: [1.21567492 1.10626207 1.27118313]
# Layer 8: [1.31434162 1.05713362 1.20893514]
# Layer 10: [0.53708422 0.45134145 0.53175118]
# Layer 11: [1.26600244 1.2117639  1.34592529]
# Layer 12: [1.22536469 1.25517766 1.28139669]
# Layer 14: [0.63817708 0.579855   0.54993886]
# Layer 15: [0.60883385 0.54367562 0.63035035]
# Layer 16: [0.63142629 0.55475446 0.56325724]
# Conv latency: [0.319610595703125, 3.1058682998021445, 1.0387621521949768, 1.9425264596939087, 0.6529245972633362, 1.1062620679537456, 1.0571336150169373, 0.451341450214386, 1.211763898531596, 1.2253646850585938, 0.5499388575553894, 0.5436756213506063, 0.5547544558842977]
# Hybrid minimum of mean layer latency: 13.785106857617697

# Results of single method:
# 'coded-3': {'min':13.057804346084595, 'mean':15.732304016749062, 'max':21.795069217681885}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-2': {'min':12.0279221534729, 'mean':15.231571833292643, 'max':21.57484006881714}
# Distributed conv idxes: [3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-1': {'min':11.274855136871338, 'mean':14.707427899042763, 'max':21.86122465133667}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# Results of mixed method:
# Layer 0: [1.57488638 1.40284145 1.22208905]
# Layer 1: [4.02579647 3.94764143 3.3843476 ]
# Layer 3: [1.20207723 1.1013335  1.12108767]
# Layer 4: [1.98877537 2.13685618 1.85736416]
# Layer 6: [0.76797688 0.71400777 0.67733753]
# Layer 7: [1.32225515 1.44050658 1.18866915]
# Layer 8: [1.23009014 1.23143222 1.45492474]
# Layer 10: [0.51308523 0.51775785 0.48691714]
# Layer 11: [1.2904308  1.07971478 1.2224655 ]
# Layer 12: [1.24375097 1.10517792 1.1925928 ]
# Layer 14: [0.59975743 0.54075011 0.5954176 ]
# Layer 15: [0.59792499 0.53883684 0.60193014]
# Layer 16: [0.60559265 0.53276596 0.57958319]
# Conv latency: [0.319610595703125, 3.384347597757975, 1.101333498954773, 1.8573641578356426, 0.6773375272750854, 1.1886691451072693, 1.2300901412963867, 0.4869171380996704, 1.0797147750854492, 1.105177919069926, 0.5407501061757406, 0.5388368368148804, 0.5327659646670023]
# Hybrid minimum of mean layer latency: 14.06809550523758

# Results of single method:
# 'coded-2': {'min':12.03791880607605, 'mean':15.09025259812673, 'max':22.280770778656006}
# Distributed conv idxes: [3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-1': {'min':11.21585464477539, 'mean':15.150781055291494, 'max':21.953298330307007}
# Distributed conv idxes: [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# Results of mixed method:
# Layer 0: [1.41633304 1.22450056]
# Layer 1: [4.08712892 3.76153394]
# Layer 3: [1.02875859 1.11142979]
# Layer 4: [2.03941884 1.88459454]
# Layer 6: [0.78866792 0.69875077]
# Layer 7: [1.16217309 1.38735672]
# Layer 8: [1.27651455 1.14693117]
# Layer 10: [0.43783331 0.48366664]
# Layer 11: [1.08886345 1.28977827]
# Layer 12: [1.15733451 1.1908412 ]
# Layer 14: [0.5601715  0.61660083]
# Layer 15: [0.55734048 0.63617069]
# Layer 16: [0.56125675 0.5983358 ]
# Conv latency: [0.319610595703125, 3.7615339358647666, 1.0287585854530334, 1.8845945398012798, 0.6987507740656534, 1.162173092365265, 1.1469311714172363, 0.437833309173584, 1.088863452275594, 1.1573345065116882, 0.5601715048154196, 0.5573404828707377, 0.5612567464510599]
# Hybrid minimum of mean layer latency: 14.390332798163097

vgg16_latency_lambda8 = np.asarray(
    [[1.5944193,  1.39893893, 1.22967013],
    [4.75798122, 3.44498706, 3.1058683 ],
    [1.20207723, 1.02875859, 1.11142979],
    [1.98877537, 2.13685618, 1.85736416],
    [0.80565492, 0.6529246,  0.72408827],
    [1.32225515, 1.44050658, 1.18866915],
    [1.31434162, 1.27651455, 1.14693117],
    [0.51308523, 0.51775785, 0.48691714],
    [1.2904308,  1.07971478, 1.2224655],
    [1.24375097, 1.10517792, 1.1925928],
    [0.63817708, 0.579855,   0.54993886],
    [0.59792499, 0.53883684, 0.60193014],
    [0.63142629, 0.55475446, 0.56325724]])

distributed_conv_layers = [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
distributed_conv_idxes = get_idxes(vgg16_conv_idxes, distributed_conv_layers)
records = vgg16_latency_lambda8[distributed_conv_idxes]
ks = records.argmin(axis=-1) + 7
print(list(ks))

[9, 8, 9, 8, 9, 9, 9, 8, 8, 9, 8, 8]


In [61]:
# lambda=10
# 'coded-3': {'min':13.004077196121216, 'mean':17.229839622974396, 'max':26.381492614746094}
# Distributed conv idxes: [1, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-2': {'min':11.819149494171143, 'mean':16.861709972222645, 'max':23.60874915122986}
# Distributed conv idxes: [3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-1': {'min':11.930824518203735, 'mean':17.2585262854894, 'max':25.891850471496582}
# Distributed conv idxes: [3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# Results of mixed method:
# Layer 0: [1.58445726 1.39060229 1.23383504]
# Layer 1: [3.69744668 4.67654314 4.32793893]
# Layer 3: [1.37601686 1.17525206 1.25800598]
# Layer 4: [2.41168632 2.16410963 2.43752003]
# Layer 6: [0.79530011 0.7626858  0.76775602]
# Layer 7: [1.55162783 1.43435721 1.72068421]
# Layer 8: [1.84443611 1.58671323 1.30426743]
# Layer 10: [0.54222852 0.55758411 0.60075255]
# Layer 11: [1.36654464 1.23163799 1.32493091]
# Layer 12: [1.34889026 1.17951973 1.35151575]
# Layer 14: [0.64617771 0.59175426 0.59859423]
# Layer 15: [0.65766925 0.5791718  0.60984651]
# Layer 16: [0.64702463 0.57759033 0.61192304]
# Conv latency: [0.319610595703125, 3.6974466840426126, 1.1752520600954692, 2.1641096274058023, 0.7626857956250509, 1.4343572060267131, 1.304267426331838, 0.5422285199165344, 1.231637994448344, 1.1795197327931721, 0.5917542576789856, 0.5791717966397604, 0.5775903264681498]
# Hybrid minimum of mean layer latency: 15.58481212457021

# 'coded-3': {'min':13.956903457641602, 'mean':20.842210849126182, 'max':28.421425580978394}
# Distributed conv idxes: [3, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-2': {'min':12.001892328262329, 'mean':16.633745551109314, 'max':25.367257118225098}
# Distributed conv idxes: [3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# 'coded-1': {'min':11.939911603927612, 'mean':16.165348370869953, 'max':23.390743017196655}
# Distributed conv idxes: [3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
# Results of mixed method:
# Layer 0: [1.58500381 1.39075377 1.2225828 ]
# Layer 1: [5.85106307 4.00890505 3.97328122]
# Layer 3: [1.55784969 1.23020548 1.07725594]
# Layer 4: [3.6460219  2.54161429 2.37016875]
# Layer 6: [0.9174189  0.88625789 0.66866652]
# Layer 7: [1.71359255 1.51698448 1.3625958 ]
# Layer 8: [1.57834508 1.4146803  1.43342127]
# Layer 10: [0.60650659 0.58866825 0.55167166]
# Layer 11: [1.22142867 1.19251049 1.30350326]
# Layer 12: [1.50852056 1.21577102 1.30203406]
# Layer 14: [0.64208523 0.55969014 0.58775244]
# Layer 15: [0.61433423 0.57475007 0.59553931]
# Layer 16: [0.64025368 0.5589174  0.59466743]
# Conv latency: [0.319610595703125, 3.973281224568685, 1.07725594441096, 2.3701687455177307, 0.6686665217081705, 1.362595796585083, 1.414680302143097, 0.5516716639200846, 1.1925104856491089, 1.2157710194587708, 0.5596901377042135, 0.5747500658035278, 0.5589174032211304]
# Hybrid minimum of mean layer latency: 15.864750007788341

vgg16_latency_lambda10 = np.asarray(
    [[1.58445726, 1.39060229, 1.23383504],
    [4.32793893, 4.00890505, 3.69744668], # What can I say
    [1.37601686, 1.17525206, 1.25800598],
    [2.41168632, 2.16410963, 2.43752003],
    [0.79530011, 0.7626858,  0.76775602],
    [1.71359255, 1.51698448, 1.3625958],
    [1.84443611, 1.58671323, 1.30426743],
    [0.60650659, 0.58866825, 0.55167166],
    [1.22142867, 1.19251049, 1.30350326],
    [1.50852056, 1.21577102, 1.30203406],
    [0.64617771, 0.59175426, 0.59859423],
    [0.65766925, 0.5791718,  0.60984651],
    [0.64702463, 0.57759033, 0.61192304]])

distributed_conv_layers = [1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]
distributed_conv_idxes = get_idxes(vgg16_conv_idxes, distributed_conv_layers)
records = vgg16_latency_lambda10[distributed_conv_idxes]
ks = records.argmin(axis=-1) + 7
print(list(ks))

[9, 8, 8, 8, 9, 9, 9, 8, 8, 8, 8, 8]


In [38]:
vgg16_layers_latency_lambdas = [vgg16_latency_lambda0, vgg16_latency_lambda2, vgg16_latency_lambda4, vgg16_latency_lambda6, vgg16_latency_lambda8, vgg16_latency_lambda10]
save_object(vgg16_layers_latency_lambdas, 'record/vgg16_layers_latency_lambdas.rec')


ResNet18

In [62]:
# lambda=0
# Results of single method:
# 'coded-3': {'min':28.04517960548401, 'mean':29.97211583455404, 'max':32.265912771224976}
# Distributed conv idxes: [3, 6, 10, 13, 17, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# 'coded-2': {'min':25.05281114578247, 'mean':26.710024615128834, 'max':28.52419924736023}
# Distributed conv idxes: [3, 6, 10, 13, 17, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# 'coded-1': {'min':23.58984375, 'mean':24.722873051961262, 'max':25.977683067321777}
# Distributed conv idxes: [3, 6, 10, 13, 17, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# Results of mixed method:
# Layer 0: [1.5913343  1.41625017 1.22667269]
# Layer 3: [3.17611067 2.75445567 2.45492613]
# Layer 6: [3.11386335 2.73351389 2.53010494]
# Layer 10: [3.15410691 2.73227117 2.56592733]
# Layer 13: [3.08927427 2.80660133 2.59909862]
# Layer 17: [2.15068247 1.91050424 1.70328764]
# Layer 20: [1.63816961 1.44702899 1.28542185]
# Layer 22: [1.94959048 1.69350372 1.46479199]
# Layer 26: [1.63060679 1.47255371 1.3012446 ]
# Layer 29: [1.63949857 1.45117146 1.27957751]
# Layer 33: [1.17800611 1.07392871 0.96341827]
# Layer 36: [1.07733238 0.99300301 0.89467249]
# Layer 38: [0.93017409 0.83483744 0.75591886]
# Layer 42: [1.06317512 1.01800865 0.91490424]
# Layer 45: [1.07908414 1.00567438 0.88901542]
# Layer 49: [0.78084459 0.63708621 0.64325051]
# Layer 52: [1.18733762 1.00310125 1.02433898]
# Layer 54: [0.44533575 0.34216736 0.34141741]
# Layer 58: [1.17509796 1.00350567 1.01707345]
# Layer 61: [1.19726364 1.0277559  1.01750066]
# Conv latency: [0.31938815116882324, 2.4549261331558228, 2.53010493516922, 2.5659273266792297, 2.5990986227989197, 1.7032876412073772, 1.2854218482971191, 0.3371391296386719, 1.301244596640269, 1.2795775135358174, 0.9634182651837667, 0.8946724931399027, 0.3606750965118408, 0.9149042367935181, 0.8890154163042704, 0.6370862126350403, 1.0031012495358784, 0.34141741196314496, 1.0035056670506795, 1.0175006588300068]
# Hybrid minimum of mean layer latency: 24.68190324306488

resnet_latency_lambda0 = np.asarray(
    [[1.5913343,  1.41625017, 1.22667269],
    [3.17611067, 2.75445567, 2.45492613],
    [3.11386335, 2.73351389, 2.53010494],
    [3.15410691, 2.73227117, 2.56592733],
    [3.08927427, 2.80660133, 2.59909862],
    [2.15068247, 1.91050424, 1.70328764],
    [1.63816961, 1.44702899, 1.28542185],
    [1.94959048, 1.69350372, 1.46479199],
    [1.63060679, 1.47255371, 1.3012446 ],
    [1.63949857, 1.45117146, 1.27957751],
    [1.17800611, 1.07392871, 0.96341827],
    [1.07733238, 0.99300301, 0.89467249],
    [0.93017409, 0.83483744, 0.75591886],
    [1.06317512, 1.01800865, 0.91490424],
    [1.07908414, 1.00567438, 0.88901542],
    [0.78084459, 0.63708621, 0.64325051],
    [1.18733762, 1.00310125, 1.02433898],
    [0.44533575, 0.34216736, 0.34141741],
    [1.17509796, 1.00350567, 1.01707345],
    [1.19726364, 1.0277559,  1.01750066]])

distributed_conv_layers = [3, 6, 10, 13, 17, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
distributed_conv_idxes = get_idxes(resnet_conv_idxes, distributed_conv_layers)
records = resnet_latency_lambda0[distributed_conv_idxes]
ks = records.argmin(axis=-1) + 7
print(list(ks))

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 9]


In [63]:
# lambda=2

# Results of single method:
# 'coded-2': {'min':25.377307653427124, 'mean':27.297936340173088, 'max':29.78143048286438}
# Distributed conv idxes: [3, 6, 10, 13, 17, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# 'coded-1': {'min':24.021623611450195, 'mean':25.076455039978027, 'max':26.73209047317505}
# Distributed conv idxes: [3, 6, 10, 13, 17, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# Results of mixed method:
# Layer 0: [1.41093    1.23168449]
# Layer 3: [2.84045284 2.65644803]
# Layer 6: [2.84654421 2.67376653]
# Layer 10: [2.85994103 2.63188795]
# Layer 13: [3.02169897 2.59186832]
# Layer 17: [1.93001654 1.76582523]
# Layer 20: [1.49892175 1.2952744 ]
# Layer 22: [1.81167495 1.61649881]
# Layer 26: [1.47693485 1.30179852]
# Layer 29: [1.47750994 1.30509174]
# Layer 33: [1.04077699 0.95377775]
# Layer 36: [1.02795714 0.86942716]
# Layer 38: [0.89669484 0.77725639]
# Layer 42: [1.01445164 0.86250633]
# Layer 45: [1.01235038 0.87285002]
# Layer 49: [0.60380012 0.6469262 ]
# Layer 52: [0.99876998 0.99258316]
# Layer 54: [0.34092174 0.35408384]
# Layer 58: [0.98485104 1.01367567]
# Layer 61: [0.97434417 1.00108637]
# Conv latency: [0.31938815116882324, 2.656448029677073, 2.673766533533732, 2.631887952486674, 2.5918683211008706, 1.7658252318700154, 1.2952743967374165, 0.3371391296386719, 1.3017985224723816, 1.3050917387008667, 0.9537777503331503, 0.8694271643956503, 0.3606750965118408, 0.8625063300132751, 0.8728500207265218, 0.6038001179695129, 0.992583155632019, 0.3409217397371928, 0.9848510424296061, 0.9743441740671793]
# Hybrid minimum of mean layer latency: 24.974715236028038

resnet_latency_lambda2 = np.asarray(
    [[1.41093,    1.23168449],
    [2.84045284, 2.65644803],
    [2.84654421, 2.67376653],
    [2.85994103, 2.63188795],
    [3.02169897, 2.59186832],
    [1.93001654, 1.76582523],
    [1.49892175, 1.2952744 ],
    [1.81167495, 1.61649881],
    [1.47693485, 1.30179852],
    [1.47750994, 1.30509174],
    [1.04077699, 0.95377775],
    [1.02795714, 0.86942716],
    [0.89669484, 0.77725639],
    [1.01445164, 0.86250633],
    [1.01235038, 0.87285002],
    [0.60380012, 0.6469262 ],
    [0.99876998, 0.99258316],
    [0.34092174, 0.35408384],
    [0.98485104, 1.01367567],
    [0.97434417, 1.00108637]])

distributed_conv_layers = [3, 6, 10, 13, 17, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
distributed_conv_idxes = get_idxes(resnet_conv_idxes, distributed_conv_layers)
records = resnet_latency_lambda2[distributed_conv_idxes]
ks = records.argmin(axis=-1) + 8
print(list(ks))

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 8, 8]


In [64]:
# lambda=4
# 'coded-3': {'min':26.618136167526245, 'mean':29.544824103514355, 'max':36.597883224487305}
# Distributed conv idxes: [3, 6, 10, 13, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# 'coded-2': {'min':24.585948944091797, 'mean':27.56151042381922, 'max':35.047091007232666}
# Distributed conv idxes: [3, 6, 10, 13, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# 'coded-1': {'min':23.05089044570923, 'mean':25.55128103494644, 'max':32.01185846328735}
# Distributed conv idxes: [3, 6, 10, 13, 17, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# Results of mixed method:
# Layer 0: [1.59135081 1.40642929 1.22925651]
# Layer 3: [3.10431147 2.80963333 2.66721741]
# Layer 6: [2.91485931 2.7938445  2.56186308]
# Layer 10: [3.02633133 2.7959882  2.64417531]
# Layer 13: [2.88466952 2.84178285 2.57967315]
# Layer 17: [2.31875978 2.23302937 2.04764771]
# Layer 20: [1.61861694 1.50175107 1.31050324]
# Layer 22: [2.15268715 2.10067447 1.64142201]
# Layer 26: [1.62419198 1.63234083 1.28085363]
# Layer 29: [1.73096561 1.63319218 1.30792447]
# Layer 33: [1.21958512 1.15408327 0.96075467]
# Layer 36: [1.02366694 0.96841812 0.92733298]
# Layer 38: [1.11458687 0.89766715 0.89300478]
# Layer 42: [1.06217317 0.9592525  0.99683291]
# Layer 45: [1.07850464 0.97041965 0.88366832]
# Layer 49: [0.77150152 0.61650155 0.65041836]
# Layer 52: [1.18250612 1.00360362 1.03316637]
# Layer 54: [0.52783285 0.40341638 0.36500055]
# Layer 58: [1.17250907 0.99342096 1.01741622]
# Layer 61: [1.17000993 1.01258679 1.04017154]
# Conv latency: [0.31938815116882324, 2.667217413584391, 2.5618630846341452, 2.6441753109296164, 2.579673151175181, 2.047647714614868, 1.3105032444000244, 0.3371391296386719, 1.2808536291122437, 1.307924469312032, 0.9607546726862589, 0.9273329774538676, 0.3606750965118408, 0.9592524965604147, 0.8836683233579, 0.6165015498797098, 1.0036036173502605, 0.34396862983703613, 0.9934209585189819, 1.0125867923100789]
# Hybrid minimum of mean layer latency: 25.398641049861908

resnet_latency_lambda4 = np.asarray(
    [[1.59135081, 1.40642929, 1.22925651],
    [3.10431147, 2.80963333, 2.66721741],
    [2.91485931, 2.7938445,  2.56186308],
    [3.02633133, 2.7959882,  2.64417531],
    [2.88466952, 2.84178285, 2.57967315],
    [2.31875978, 2.23302937, 2.04764771],
    [1.61861694, 1.50175107, 1.31050324],
    [2.15268715, 2.10067447, 1.64142201],
    [1.62419198, 1.63234083, 1.28085363],
    [1.73096561, 1.63319218, 1.30792447],
    [1.21958512, 1.15408327, 0.96075467],
    [1.02366694, 0.96841812, 0.92733298],
    [1.11458687, 0.89766715, 0.89300478],
    [1.06217317, 0.9592525,  0.99683291],
    [1.07850464, 0.97041965, 0.88366832],
    [0.77150152, 0.61650155, 0.65041836],
    [1.18250612, 1.00360362, 1.03316637],
    [0.52783285, 0.40341638, 0.36500055],
    [1.17250907, 0.99342096, 1.01741622],
    [1.17000993, 1.01258679, 1.04017154]])

distributed_conv_layers = [3, 6, 10, 13, 17, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
distributed_conv_idxes = get_idxes(resnet_conv_idxes, distributed_conv_layers)
records = resnet_latency_lambda4[distributed_conv_idxes]
ks = records.argmin(axis=-1) + 7
print(list(ks))

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 8, 8, 8, 8]


In [65]:
# lambda=6
# 'coded-3': {'min':27.371126174926758, 'mean':34.75535567601522, 'max':49.32160472869873}
# Distributed conv idxes: [3, 6, 13, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# 'coded-2': {'min':24.8619487285614, 'mean':31.022449930508927, 'max':45.525224924087524}
# Distributed conv idxes: [3, 6, 10, 13, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# 'coded-1': {'min':23.529942750930786, 'mean':29.31320383151372, 'max':38.397050857543945}
# Distributed conv idxes: [3, 6, 10, 13, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# Results of mixed method:
# Layer 0: [1.56894177 1.40550113 1.22925115]
# Layer 3: [3.62696284 3.19379461 2.89218777]
# Layer 6: [3.58639528 3.18116089 2.80736462]
# Layer 10: [3.90845988 3.03170904 2.84103479]
# Layer 13: [3.64084103 3.05894214 3.16037607]
# Layer 17: [2.86061873 2.81787703 2.61251869]
# Layer 20: [2.02613356 1.77183535 1.45325343]
# Layer 22: [2.91019342 2.39725361 1.98833825]
# Layer 26: [1.91028446 1.55310102 1.53667263]
# Layer 29: [1.73335991 1.92011056 1.76185012]
# Layer 33: [1.61951904 1.62675526 1.37167627]
# Layer 36: [1.05885196 1.08659164 1.04235192]
# Layer 38: [1.52309922 1.32367857 1.04825219]
# Layer 42: [1.25048429 1.094489   0.94325014]
# Layer 45: [1.42880181 1.06834364 1.05016883]
# Layer 49: [0.85000587 0.85425117 0.77124689]
# Layer 52: [1.19978579 1.02684917 1.15530566]
# Layer 54: [0.55867316 0.51167969 0.46224988]
# Layer 58: [1.2053869  1.00759145 1.14502931]
# Layer 61: [1.20780269 1.08738633 1.12725504]
# Conv latency: [0.31938815116882324, 2.892187774181366, 2.8073646227518716, 2.841034789880117, 3.058942139148712, 2.6125186880429587, 1.4532534281412761, 0.3371391296386719, 1.5366726318995159, 1.7333599130312602, 1.37167626619339, 1.0423519213994343, 0.3606750965118408, 0.9432501395543417, 1.0501688321431477, 0.7712468902269999, 1.0268491705258687, 0.34396862983703613, 1.007591446240743, 1.0873863299687703]
# Hybrid minimum of mean layer latency: 28.877516627311703

resnet_latency_lambda6 = np.asarray(
    [[1.56894177, 1.40550113, 1.22925115],
    [3.62696284, 3.19379461, 2.89218777],
    [3.58639528, 3.18116089, 2.80736462],
    [3.90845988, 3.03170904, 2.84103479],
    [3.64084103, 3.05894214, 3.16037607],
    [2.86061873, 2.81787703, 2.61251869],
    [2.02613356, 1.77183535, 1.45325343],
    [2.91019342, 2.39725361, 1.98833825],
    [1.91028446, 1.55310102, 1.53667263],
    [1.73335991, 1.92011056, 1.76185012],
    [1.61951904, 1.62675526, 1.37167627],
    [1.05885196, 1.08659164, 1.04235192],
    [1.52309922, 1.32367857, 1.04825219],
    [1.25048429, 1.094489,   0.94325014],
    [1.42880181, 1.06834364, 1.05016883],
    [0.85000587, 0.85425117, 0.77124689],
    [1.19978579, 1.02684917, 1.15530566],
    [0.55867316, 0.51167969, 0.46224988],
    [1.2053869,  1.00759145, 1.14502931],
    [1.20780269, 1.08738633, 1.12725504]])

distributed_conv_layers = [3, 6, 10, 13, 17, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
distributed_conv_idxes = get_idxes(resnet_conv_idxes, distributed_conv_layers)
records = resnet_latency_lambda6[distributed_conv_idxes]
ks = records.argmin(axis=-1) + 7
print(list(ks))

[9, 9, 9, 8, 9, 9, 9, 7, 9, 9, 9, 9, 9, 8, 8, 8]


In [66]:
# lambda=8
# Results of single method:
# 'coded-3': {'min':27.96196746826172, 'mean':39.4187124768893, 'max':56.62533688545227}
# Distributed conv idxes: [10, 13, 20, 26, 29, 36, 42, 45, 49, 52, 58, 61]
# 'coded-2': {'min':25.722853422164917, 'mean':36.56455955902736, 'max':49.856924533843994}
# Distributed conv idxes: [3, 13, 20, 26, 29, 36, 42, 45, 49, 52, 58, 61]
# 'coded-1': {'min':25.105061054229736, 'mean':34.62424453099569, 'max':49.03157448768616}
# Distributed conv idxes: [3, 6, 10, 13, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# Results of mixed method:
# Layer 0: [1.57909721 1.40545976 1.23508712]
# Layer 3: [4.18149821 3.36240105 3.45525545]
# Layer 6: [4.41934204 4.99411358 3.64716874]
# Layer 10: [3.43997763 3.63586082 3.8758507 ]
# Layer 13: [3.73440639 3.6938534  3.59425032]
# Layer 17: [3.77110066 2.79083389 2.87300696]
# Layer 20: [2.45395726 2.09440152 1.65168661]
# Layer 22: [3.5295824  2.55010351 2.51802965]
# Layer 26: [2.38051057 2.05892418 1.97569962]
# Layer 29: [2.64346457 1.96443564 1.76825541]
# Layer 33: [2.48084178 2.1455792  1.60027627]
# Layer 36: [1.25733618 1.25584078 1.26042179]
# Layer 38: [1.70408734 1.66043009 1.29658709]
# Layer 42: [1.45775455 1.35407253 1.36492489]
# Layer 45: [1.51836252 1.30992347 1.23000304]
# Layer 49: [1.16693739 0.88243151 0.9955859 ]
# Layer 52: [1.26235392 1.11068573 1.25884682]
# Layer 54: [0.72691778 0.62916762 0.66446382]
# Layer 58: [1.22500076 1.1986168  1.19709273]
# Layer 61: [1.26401736 1.07092383 1.23425764]
# Conv latency: [0.31938815116882324, 3.3624010483423867, 3.647168735663096, 3.4399776260058084, 3.5942503213882446, 2.79083389043808, 1.6516866087913513, 0.3371391296386719, 1.9756996234258015, 1.7682554125785828, 1.600276271502177, 1.25584077835083, 0.3606750965118408, 1.3540725310643513, 1.230003039042155, 0.8824315071105957, 1.1106857260068257, 0.34396862983703613, 1.1970927317937214, 1.0709238251050313]
# Hybrid minimum of mean layer latency: 33.57326132059097

# Results of single method:
# 'coded-3': {'min':27.40676236152649, 'mean':38.85847030083338, 'max':57.05828619003296}
# Distributed conv idxes: [10, 20, 26, 36, 42, 45, 49, 52, 58, 61]
# 'coded-2': {'min':25.775699138641357, 'mean':36.65564205249151, 'max':52.19920063018799}
# Distributed conv idxes: [3, 20, 26, 29, 36, 42, 45, 49, 52, 58, 61]
# 'coded-1': {'min':24.683436155319214, 'mean':33.83119583129882, 'max':48.491368770599365}
# Distributed conv idxes: [3, 6, 13, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# Results of mixed method:
# Layer 0: [1.587982   1.40142822 1.28197875]
# Layer 3: [3.93578978 3.48903261 3.36562363]
# Layer 6: [3.68102274 4.50095169 3.16167309]
# Layer 10: [3.9657733  4.10610829 3.78129415]
# Layer 13: [5.03678022 3.99211413 3.63627867]
# Layer 17: [3.17437659 3.98936796 2.65876242]
# Layer 20: [2.25284658 1.96434029 1.84867088]
# Layer 22: [3.28444972 3.60186535 2.2582674 ]
# Layer 26: [2.14977386 1.96470316 2.09177395]
# Layer 29: [2.36053787 2.19893573 1.89255784]
# Layer 33: [2.14759894 1.91634456 1.60288711]
# Layer 36: [1.26916919 1.18175759 1.1589213 ]
# Layer 38: [1.70409785 1.69368702 1.33800536]
# Layer 42: [1.37092038 1.29184423 1.18359754]
# Layer 45: [1.24650274 1.35450878 1.34988834]
# Layer 49: [1.07903481 0.85452318 0.96084042]
# Layer 52: [1.26909471 1.09267773 1.15675296]
# Layer 54: [0.88333563 0.80583415 0.5665858 ]
# Layer 58: [1.20416965 1.09917315 1.15108679]
# Layer 61: [1.35650418 1.11567559 1.18892509]
# Conv latency: [0.31938815116882324, 3.365623633066813, 3.161673088868459, 3.7812941471735635, 3.636278669039408, 2.6587624152501426, 1.8486708799997966, 0.3371391296386719, 1.9647031625111897, 1.8925578395525615, 1.6028871138890584, 1.1589213013648987, 0.3606750965118408, 1.1835975448290508, 1.2465027372042339, 0.8545231819152832, 1.0926777323087056, 0.34396862983703613, 1.0991731484731038, 1.1156755884488423]
# Hybrid minimum of mean layer latency: 33.305183827877045

# Results of single method:
# 'coded-2': {'min':26.01596212387085, 'mean':36.13778324921926, 'max':53.39074659347534}
# Distributed conv idxes: [6, 10, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# 'coded-1': {'min':24.823166847229004, 'mean':33.81694052616755, 'max':51.748284578323364}
# Distributed conv idxes: [3, 10, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# Results of mixed method:
# Layer 0: [1.41843482 1.22805383]
# Layer 3: [4.14887224 3.5006065 ]
# Layer 6: [3.86023732 3.56286971]
# Layer 10: [4.05343384 3.31758904]
# Layer 13: [3.65244746 3.76167081]
# Layer 17: [3.84859971 2.55150352]
# Layer 20: [1.75916807 1.8655057 ]
# Layer 22: [3.43142301 2.64133908]
# Layer 26: [1.93751162 2.0987497 ]
# Layer 29: [2.12567437 1.72300263]
# Layer 33: [1.8841781  1.54675657]
# Layer 36: [1.24917275 1.2389217 ]
# Layer 38: [1.74367958 1.19267029]
# Layer 42: [1.27825377 1.19042746]
# Layer 45: [1.44310214 1.26000621]
# Layer 49: [0.7940862  1.01099968]
# Layer 52: [1.103338   1.18583554]
# Layer 54: [0.56358323 0.63433377]
# Layer 58: [1.092504   1.16541684]
# Layer 61: [1.22285201 1.19541728]
# Conv latency: [0.31938815116882324, 3.5006064971288047, 3.5628697077433267, 3.317589044570923, 3.652447462081909, 2.551503519217173, 1.7591680685679119, 0.3371391296386719, 1.9375116229057312, 1.7230026324590046, 1.5467565655708313, 1.2389217019081116, 0.3606750965118408, 1.1904274622599285, 1.2600062092145283, 0.7940861980120341, 1.1033380031585693, 0.34396862983703613, 1.0925040046374004, 1.1954172849655151]
# Hybrid minimum of mean layer latency: 33.06781762838364

resnet_latency_lambda8 = np.asarray(
    [[1.57909721, 1.40545976, 1.23508712],
    [4.18149821, 3.36240105, 3.45525545],
    [4.41934204, 3.86023732, 3.56286971],
    [3.43997763, 3.63586082, 3.8758507 ],
    [3.73440639, 3.65244746, 3.76167081],
    [3.77110066, 3.84859971, 2.55150352],
    [2.45395726, 2.09440152, 1.65168661],
    [3.28444972, 3.60186535, 2.2582674 ],
    [2.38051057, 2.05892418, 1.97569962],
    [2.64346457, 1.96443564, 1.76825541],
    [2.48084178, 1.8841781,  1.54675657],
    [1.26916919, 1.18175759, 1.1589213 ],
    [1.70408734, 1.66043009, 1.19267029],
    [1.37092038, 1.29184423, 1.18359754],
    [1.51836252, 1.30992347, 1.23000304],
    [1.07903481, 0.85452318, 0.96084042],
    [1.26909471, 1.09267773, 1.15675296],
    [0.88333563, 0.80583415, 0.5665858 ],
    [1.20416965, 1.09917315, 1.15108679],
    [1.35650418, 1.11567559, 1.18892509]])

distributed_conv_layers = [3, 6, 10, 13, 17, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
distributed_conv_idxes = get_idxes(resnet_conv_idxes, distributed_conv_layers)
records = resnet_latency_lambda8[distributed_conv_idxes]
ks = records.argmin(axis=-1) + 7
print(list(ks))

[8, 9, 7, 8, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8]


In [67]:
# lambda=10

# Results of single method:
# 'coded-3': {'min':29.342271089553833, 'mean':47.476320008436836, 'max':64.39616060256958}
# Distributed conv idxes: [20, 36, 42, 45, 49, 52, 58, 61]
# 'coded-2': {'min':27.565308094024658, 'mean':41.858186165491745, 'max':57.96237277984619}
# Distributed conv idxes: [20, 26, 29, 36, 42, 45, 49, 52, 58, 61]
# 'coded-1': {'min':26.02274990081787, 'mean':38.08953764041265, 'max':56.04798936843872}
# Distributed conv idxes: [20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# Results of mixed method:
# Layer 0: [1.54894042 1.37192219 1.22900784]
# Layer 3: [5.92287364 4.66463552 3.96019123]
# Layer 6: [5.25124443 5.507967   4.23079216]
# Layer 10: [5.1546373  4.682604   3.84395409]
# Layer 13: [5.01008366 4.39188735 4.06731774]
# Layer 17: [4.38003472 5.73244347 3.56357596]
# Layer 20: [2.36502415 1.94149993 2.24653365]
# Layer 22: [4.7312628  4.21657032 2.88532956]
# Layer 26: [3.65760732 2.44367172 2.07948921]
# Layer 29: [2.84684849 2.32643427 1.95378613]
# Layer 33: [2.23835931 1.97544386 1.77485089]
# Layer 36: [1.72986964 1.46756013 1.22078526]
# Layer 38: [2.42742087 2.09180158 1.69479434]
# Layer 42: [1.67020096 1.5261846  1.40623705]
# Layer 45: [1.66468378 1.72260191 1.43362017]
# Layer 49: [1.32893658 1.01835446 1.35359563]
# Layer 52: [1.4175303  1.18978818 1.29404962]
# Layer 54: [0.85527295 0.67825651 0.65675457]
# Layer 58: [1.3610298  1.27556833 1.31761299]
# Layer 61: [1.32443931 1.19103356 1.37377046]
# Conv latency: [0.31938815116882324, 3.960191229979197, 4.230792164802551, 3.843954086303711, 4.06731774409612, 2.891289710998535, 1.941499928633372, 0.3371391296386719, 2.079489211241404, 1.9537861347198486, 1.774850885073344, 1.2207852602005005, 0.3606750965118408, 1.406237045923869, 1.4336201747258503, 1.0183544556299846, 1.1897881825764973, 0.34396862983703613, 1.2755683263142903, 1.1910335620244343]
# Hybrid minimum of mean layer latency: 37.120219747225455

# Results of single method:
# 'coded-2': {'min':29.478199005126953, 'mean':40.92967297633489, 'max':56.51185178756714}
# Distributed conv idxes: [6, 20, 26, 29, 36, 42, 45, 49, 52, 58, 61]
# 'coded-1': {'min':26.301873207092285, 'mean':38.500385661919914, 'max':54.94946599006653}
# Distributed conv idxes: [20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
# Results of mixed method:
# Layer 0: [1.41229248 1.22966703]
# Layer 3: [4.91295463 3.99134445]
# Layer 6: [4.23434442 4.38893692]
# Layer 10: [4.86297035 4.14420219]
# Layer 13: [3.69750152 4.17339337]
# Layer 17: [4.93964644 4.12329404]
# Layer 20: [2.28044661 2.19609175]
# Layer 22: [5.00773046 3.29343329]
# Layer 26: [2.34425559 1.8740961 ]
# Layer 29: [2.4187598  2.00942508]
# Layer 33: [2.42034495 1.8819199 ]
# Layer 36: [1.3985919  1.31008341]
# Layer 38: [2.01350254 1.92975001]
# Layer 42: [1.40384428 1.39610281]
# Layer 45: [1.54958763 1.43984709]
# Layer 49: [1.16133328 1.04400446]
# Layer 52: [1.20635285 1.36666971]
# Layer 54: [0.73216661 0.8400045 ]
# Layer 58: [1.19834427 1.33321941]
# Layer 61: [1.30708955 1.41809765]
# Conv latency: [0.31938815116882324, 3.991344451904297, 4.23434442281723, 4.14420219262441, 3.697501520315806, 2.891289710998535, 2.1960917512575784, 0.3371391296386719, 1.8740960955619812, 2.009425083796183, 1.8819199005762737, 1.3100834091504414, 0.3606750965118408, 1.3961028059323628, 1.4398470918337505, 1.044004460175832, 1.2063528498013814, 0.34396862983703613, 1.1983442703882854, 1.3070895473162334]
# Hybrid minimum of mean layer latency: 37.46370120843251

# Results of single method:
# 'coded-2': {'min':26.613318920135498, 'mean':42.327236394087464, 'max':60.47470831871033}
# Distributed conv idxes: [20, 26, 29, 36, 42, 45, 49, 52, 58, 61]
# 'coded-1': {'min':26.608710050582886, 'mean':40.47234014670054, 'max':57.26929593086243}
# Distributed conv idxes: [6, 13, 20, 26, 29, 36, 42, 45, 49, 52, 58, 61]
# Results of mixed method:
# Layer 0: [1.40767248 1.22401633]
# Layer 3: [4.15863067 4.46784715]
# Layer 6: [5.4773739  4.35778079]
# Layer 10: [4.36081469 4.13440112]
# Layer 13: [4.5153361  3.80956757]
# Layer 17: [4.19182229 3.54510812]
# Layer 20: [2.33867729 2.48521495]
# Layer 22: [3.49839516 3.27188186]
# Layer 26: [2.60124826 2.55524552]
# Layer 29: [2.69611526 2.55112189]
# Layer 33: [2.41433213 2.24486212]
# Layer 36: [1.65795006 1.54477443]
# Layer 38: [1.49769473 2.22624938]
# Layer 42: [1.5100864  1.38239706]
# Layer 45: [1.42895744 1.38023869]
# Layer 49: [1.05660594 1.24717557]
# Layer 52: [1.22506487 1.27568376]
# Layer 54: [0.71577519 0.90677991]
# Layer 58: [1.22926692 1.25721508]
# Layer 61: [1.12382511 1.24586308]
# Conv latency: [0.31938815116882324, 4.158630669116974, 4.357780794302623, 4.134401122728984, 3.8095675706863403, 2.891289710998535, 2.3386772871017456, 0.3371391296386719, 2.555245518684387, 2.5511218905448914, 2.244862119356791, 1.5447744329770405, 0.3606750965118408, 1.3823970556259155, 1.380238691965739, 1.0566059350967407, 1.2250648736953735, 0.34396862983703613, 1.229266921679179, 1.1238251129786174]
# Hybrid minimum of mean layer latency: 39.625411351521805

resnet_latency_lambda10 = np.asarray(
    [[1.54894042, 1.37192219, 1.22900784],
    [5.92287364, 4.15863067, 4.46784715],
    [5.25124443, 4.23434442, 4.38893692],
    [5.1546373,  4.36081469, 4.13440112],
    [5.01008366, 3.69750152, 4.17339337],
    [4.38003472, 4.19182229, 3.54510812],
    [2.36502415, 1.94149993, 2.24653365],
    [4.7312628,  3.49839516, 3.27188186],
    [3.65760732, 2.34425559, 1.8740961 ],
    [2.84684849, 2.32643427, 1.95378613],
    [2.23835931, 1.97544386, 1.77485089],
    [1.72986964, 1.46756013, 1.22078526],
    [2.42742087, 2.09180158, 1.69479434],
    [1.67020096, 1.5261846,  1.40623705],
    [1.66468378, 1.72260191, 1.43362017],
    [1.32893658, 1.01835446, 1.35359563],
    [1.4175303,  1.18978818, 1.29404962],
    [0.85527295, 0.67825651, 0.65675457],
    [1.3610298,  1.27556833, 1.31761299],
    [1.32443931, 1.19103356, 1.37377046]])

distributed_conv_layers = [3, 6, 10, 13, 20, 26, 29, 33, 36, 42, 45, 49, 52, 58, 61]
distributed_conv_idxes = get_idxes(resnet_conv_idxes, distributed_conv_layers)
records = resnet_latency_lambda8[distributed_conv_idxes]
ks = records.argmin(axis=-1) + 7
print(list(ks))

[8, 9, 7, 8, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8]


In [45]:
resnet_latency_lambdas = [resnet_latency_lambda0, resnet_latency_lambda2, resnet_latency_lambda4, resnet_latency_lambda6, resnet_latency_lambda8, resnet_latency_lambda10]
save_object(resnet_latency_lambdas, 'record/resnet_layers_latency_lambdas.rec')